# Affordability

Income-to-Rent Ratio, IRR


In [215]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from sklearn.preprocessing import MinMaxScaler

In [216]:
# load data
df = pd.read_parquet("../data/curated/features_domain.parquet") 

In [217]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [218]:
# Display the first 5 rows of the dataframe
df.head()

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,index_right,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry_proj,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,mean_stores,total_stores,count_shopping_centres,SA2 code_x,Region,ERP_2021,ERP_2026,ERP_2031,ERP_2036,PNPD_2021,PNPD_2026,PNPD_2031,PNPD_2036,POPD_2021,POPD_2026,POPD_2031,POPD_2036,OPD_2021,OPD_2026,OPD_2031,OPD_2036,HHS_2021,HHS_2026,HHS_2031,HHS_2036,SPD_2021,SPD_2026,SPD_2031,SPD_2036,OCC_2021,OCC_2026,OCC_2031,OCC_2036,population_growth_2021_2026,population_growth_2026_2031,population_growth_2031_2036,total_population_growth,Facility Count,erp_2022_no.,erp_2023_no.,erp_change_no.,erp_change_percentage,pop_change_natural_increase,pop_change_net_internal_migration,pop_change_oversea_migration,area_km2,population_density_persons_per_km2_2023,property_sa2_name,nearest_parkres_distance,nearest_parkres_name,parkres_sa2_name,closest_school,distance_to_closest_school,closest_independent_school,distance_to_closest_independent_school,total_value_of_items,total_number_of_items,crime_frequency,SA2 code_y,SA2 name,Earners,Median age of earners,Sum,Median,Mean,P80/P20,P80/P50,P20/P50,P10/P50,Gini coefficient,Top 1%,Top 5%,Top 10%,Lowest Quartile,Second Quartile,Third Quartile,Highest Quartile
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.774273,145.181126,4.0,3.0,2.0,9125.0,1400.0,POINT (145.1811258 -37.77427300000001),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989259.4173701586 -4291898.086618988),Donvale,0,19900,5.6596,25.93487,H5063,1.920280,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,3541.030,Mullum Mullum Park,Donvale - Park Orchards,1759,0.345483,1724,1.086111,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
1,https://www.domain.com.au/7-pine-ridge-donvale...,$750 per week,"7 Pine Ridge, Donvale VIC 3111",House,-37.791251,145.175649,4.0,2.0,0.0,3259.0,750.0,POINT (145.1756489 -37.7912513),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (988607.8235432974 -4293742.529802319),Donvale,1,19899,4.6923,23.77997,H5063,0.384624,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,2536.544,Mullum Mullum Park,Donvale - Park Orchards,5019,0.986444,1724,2.472361,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
2,https://www.domain.com.au/20-mulsanne-way-donv...,$1300 per week,"20 Mulsanne Way, Donvale VIC 3111",House,-37.797232,145.181264,5.0,2.0,2.0,5649.0,1300.0,POINT (145.1812636 -37.7972323),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Man

In [219]:
# check median income for whole dataset
df['Median'].describe()

count     9565.000000
mean     56147.471720
std      11560.078243
min      23341.000000
25%      49545.000000
50%      55990.000000
75%      64287.000000
max      81648.000000
Name: Median, dtype: float64

In [220]:
# keep only the columns that are needed  sa2_code, sa2_name, property_type, extracted_price, Median, ERP_2021, ERP_2026, OCC_2021 and OCC_2026, population_growth_2021_2026, population_density_persons_per_km2_2023, 
# Mean, Top 10%	, Median, and Gini coefficient
df = df[['sa2_code', 'sa2_name', 'property_type', 'extracted_price', 'ERP_2021', 'ERP_2026', 'OCC_2021', 'OCC_2026', 'population_growth_2021_2026', 'population_density_persons_per_km2_2023', 'Median', 'Mean', 'Top 10%', 'Gini coefficient']]


In [221]:
# show the first 5 rows of the dataframe
df.head()

,sa2_code,sa2_name,property_type,extracted_price,ERP_2021,ERP_2026,OCC_2021,OCC_2026,population_growth_2021_2026,population_density_persons_per_km2_2023,Median,Mean,Top 10%,Gini coefficient
0,211021261,Donvale - Park Orchards,Townhouse,1400.0,16410.0,16758.344317,0.967246,0.96,2.122756,793.7,53881,83728,39.7,0.555
1,211021261,Donvale - Park Orchards,House,750.0,16410.0,16758.344317,0.967246,0.96,2.122756,793.7,53881,83728,39.7,0.555
2,211021261,Donvale - Park Orchards,House,1300.0,16410.0,16758.344317,0.967246,0.96,2.122756,793.7,53881,83728,39.7,0.555
3,211021261,Donvale - Park Orchards,House,825.0,16410.0,16758.344317,0.967246,0.96,2.122756,793.7,53881,83728,39.7,0.555
4,211021261,Donvale - Park Orchards,Townhouse,680.0,16410.0,16758.344317,0.967246,0.96,2.122756,793.7,53881,83728,39.7,0.555


In [222]:
# Calculate Income-to-Price Ratio (IPR)
# # IPR measures the affordability of housing relative to income. Higher values indicate better affordability.
df['IPR'] = df['Median'] / df['extracted_price']

# Calculate Housing Pressure Index (HPI)
# HPI measures the availability of housing. Higher values indicate less pressure on housing supply, which means better affordability.
df['HPI'] = 1 - (df['OCC_2021'] / df['ERP_2021'])

# Calculate Population Density Inverse (PDI)
# PDI is the inverse of population density. Lower population density can imply more affordable housing options.
df['PDI'] = 1 / df['population_density_persons_per_km2_2023']

# Calculate Gini Inequality Inverse (GII)
# GII represents income equality. Higher values indicate less income inequality, which is generally associated with better affordability.
df['GII'] = 1 - df['Gini coefficient']


In [223]:
# Select the newly created columns for affordability calculation
affordability_features = ['IPR', 'HPI', 'PDI', 'GII']

In [224]:
# remove inf values
df = df.replace([np.inf, -np.inf], np.nan)

In [225]:
# Standardize features using Min-Max scaling
scaler = MinMaxScaler()
df[affordability_features] = scaler.fit_transform(df[affordability_features])

In [226]:
# Define weights for each feature
weights = {'IPR': 0.4, 'HPI': 0.3, 'PDI': 0.2, 'GII': 0.1}

# Calculate Affordability Score
# The Affordability Score is a weighted sum of all standardized features, where higher scores indicate better affordability
df['Affordability_Score'] = (
    weights['IPR'] * df['IPR'] +
    weights['HPI'] * df['HPI'] +
    weights['PDI'] * df['PDI'] +
    weights['GII'] * df['GII']
)

In [227]:
# Sort by Affordability Score (higher is better)
df = df.sort_values(by='Affordability_Score', ascending=False)

In [229]:
# Display the top 10 most affordable SA2 areas
df[['sa2_name', 'Affordability_Score']].head(10)

,sa2_name,Affordability_Score
4346,Maribyrnong,0.778495
2826,Hoppers Crossing - North,0.751094
1863,Whittlesea,0.669028
1865,Whittlesea,0.669028
4018,Manor Lakes - Quandong,0.631148
4020,Werribee - West,0.610110
6597,Point Cook - South,0.605234
1864,Mickleham - Yuroke,0.594670
1866,Mickleham - Yuroke,0.594670
5160,Yarra Valley,0.585904


In [230]:
# Save the results to a new CSV file
df.to_csv('sa2_affordability_scores.csv', index=False)